In [9]:
import pandas as pd
import numpy as np
from joblib import dump, load

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

Helper Functions

In [18]:
# initial random seed
seed = 42

In [2]:
def compute_metrics(y_true, y_pred):
    """Compute RMSE, MAE, and R2."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

In [3]:
def compute_stability(model, X, y, noise_level=0.01, n_trials=5):
    """
    Add Gaussian noise to numeric features and measure average relative RMSE change.
    noise_level is fraction of std-dev of each feature.
    """
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    base_rmse = np.sqrt(mean_squared_error(y, model.predict(X)))
    rel_changes = []
    
    for _ in range(n_trials):
        Xp = X.copy()
        noise = np.random.normal(0, noise_level * Xp[numeric_cols].std(), 
                                 size=Xp[numeric_cols].shape)
        Xp[numeric_cols] += noise
        rp = model.predict(Xp)
        rmse_p = np.sqrt(mean_squared_error(y, rp))
        rel_changes.append((rmse_p - base_rmse) / base_rmse)
    
    # Return average relative change (lower = more stable)
    return np.mean(rel_changes)

# Model Definition

In [ ]:
models = {
    'Ridge': Ridge(random_state=42, alpha=1.0), 
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=seed),
    'XGBoost': XGBRegressor(n_estimators=100, use_label_encoder=False, eval_metric='rmse', random_state=seed),
    'MLP': MLPRegressor(hidden_layer_sizes=(64, 64),    # implemented a simple MLP with sklearn for easy compatibility with the rest of the code
                        activation='relu',
                        solver='adam',
                        max_iter=200,
                        random_state=42)
}

In [ ]:
param_grids = {
    'Ridge': {'alpha': [0.01, 0.1, 1.0, 10.0]},
    'RandomForest': {'n_estimators': [100, 200],
                     'max_depth': [None, 10, 20]},
    'XGBoost': {'n_estimators': [100, 200],
                'learning_rate': [0.01, 0.1],
                'max_depth': [3, 6]},
    'MLP': {'hidden_layer_sizes': [(64,), (64, 64)],
            'alpha': [1e-4, 1e-3],
            'learning_rate_init': [1e-3, 1e-2],
            'max_iter': [200]}
}

In [6]:
datasets = load('..\data\experimental\experiment_datasets_2.joblib')  # Load datasets from joblib file

# Manual Training

In [6]:
train_df = datasets['baselines']['full']['within_sample']['within_sample']['train']
test_df = datasets['baselines']['full']['within_sample']['within_sample']['test']

train_df.shape, test_df.shape

((291, 82), (73, 82))

In [7]:
X_train = train_df.drop(columns=['totalEsg'])
y_train = train_df['totalEsg']
X_test  = test_df.drop(columns=['totalEsg'])
y_test  = test_df['totalEsg']

In [8]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((291, 81), (291,), (73, 81), (73,))

In [9]:
model = models['Ridge']  # Choose the model you want to use

In [10]:
model

Ridge(random_state=42)

In [11]:
model.fit(X_train, y_train)

l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=5.17923e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge(random_state=42)

In [12]:
# predict
y_pred = model.predict(X_test)

In [13]:
# compute metrics
rmse, mae, r2 = compute_metrics(y_test, y_pred)
stability = compute_stability(model, X_test, y_test,
                            noise_level=0.01, n_trials=5)

In [14]:
manual_results = {
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'Stability': stability
        }

In [15]:
pd.DataFrame(manual_results, index=[0])

,RMSE,MAE,R2,Stability
0,44.795695,15.571981,-32.245542,-0.001084


# Iterative Training

In [ ]:
def process_dataset(train_df, test_df, models, scenario_info):
    """
    Process a single dataset with all models and return results.
    
    Parameters:
        train_df (pd.DataFrame): Training data
        test_df (pd.DataFrame): Test data
        models (dict): Dictionary of models to evaluate
        scenario_info (dict): Dictionary containing scenario metadata
    """
    temp_results = []
    
    # Separate features and target
    X_train = train_df.drop(columns=['totalEsg'])
    y_train = train_df['totalEsg']
    X_test = test_df.drop(columns=['totalEsg'])
    y_test = test_df['totalEsg']
    
    print(f"Data shapes - Train: {train_df.shape}, Test: {test_df.shape}")
    
    # Train and evaluate models
    for model_name, model in models.items():
        # Train and predict
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # Compute metrics
        rmse, mae, r2 = compute_metrics(y_test, y_pred)
        stability = compute_stability(model, X_test, y_test, 
                                   noise_level=0.01, n_trials=5)
        
        # Store results
        temp_results.append({
            **scenario_info,
            'model': model_name,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'Stability': stability
        })
    
    return temp_results

In [6]:
datasets['baselines']['full']['region_holdout']['europe_n_central_asia']['train'].shape

(275, 82)

In [7]:
datasets['baselines']['full']['region_holdout']['europe_n_central_asia'].keys()

dict_keys(['train', 'test', 'train_size', 'test_size'])

In [ ]:
results = []
n_runs = 5 # number of runs for statistical comparison

In [ ]:
%%time
for scenario_type, scenario_dict in datasets.items():        # e.g., 'baselines', 'diversified'
    for scenario_name, splits in scenario_dict.items():      # e.g., 'full', 'constrained', 'max_balanced', etc.
        for split_type, data_group in splits.items():        # 'within_sample', 'region_holdout', 'size_holdout'
            if split_type == 'original_data':
                continue  # Skip original data entries
            
            # if split_type == 'region_holdout':
            #     continue # TEMP TO CHECK SMTH

            # For within-sample, we have one group; for others, multiple contexts
            # contexts = {'within_sample': data_group} if split_type == 'within_sample' else data_group

            # for context_name, data in contexts.items():
            #     print(f"Scenario: {scenario_type}, {scenario_name}, Split: {split_type}, Context: {context_name}")

            # Handle different split types
            if split_type == 'within_sample':
                # For within_sample, data structure is one level deeper
                data = data_group['within_sample']
                print(f"Scenario: {scenario_type}, {scenario_name}, Split: {split_type}")
                if 'train' not in data or 'test' not in data:
                    continue
                    
                train_df = data['train']
                test_df = data['test']

                print(train_df.shape, test_df.shape)

                # Separate features and target
                X_train = train_df.drop(columns=['totalEsg'])
                y_train = train_df['totalEsg']
                X_test  = test_df.drop(columns=['totalEsg'])
                y_test  = test_df['totalEsg']
                
                # Train and evaluate models
                for model_name, model in models.items():
                    # Prepare grid search
                    grid = GridSearchCV(
                        estimator=model,
                        param_grid=param_grids[model_name],
                        cv=3,
                        scoring='neg_root_mean_squared_error',
                        n_jobs=-1
                    )
                    # Tune on training data
                    grid.fit(X_train, y_train)
                    best_model = grid.best_estimator_

                    # train
                    # model.fit(X_train, y_train)


                    # predict
                    # y_pred = model.predict(X_test)
                    y_pred = best_model.predict(X_test)

                    # compute metrics
                    rmse, mae, r2 = compute_metrics(y_test, y_pred)
                    stability = compute_stability(best_model, X_test, y_test,
                                                noise_level=0.01, n_trials=5)
                    
                    results.append({
                        'scenario_type': scenario_type,
                        'scenario': scenario_name,
                        'split': split_type,
                        'context': 'within_sample',
                        'model': model_name,
                        'RMSE': rmse,
                        'MAE': mae,
                        'R2': r2,
                        'Stability': stability
                    })
            
            else:
                # For region_holdout and size_holdout, process each context
                for context_name, data in data_group.items():
                    print(f"Scenario: {scenario_type}, {scenario_name}, Split: {split_type}, Context: {context_name}")
                    if 'train' not in data or 'test' not in data:
                        continue
                        
                    train_df = data['train']
                    test_df = data['test']

                    print(train_df.shape, test_df.shape)

                    # Separate features and target
                    X_train = train_df.drop(columns=['totalEsg'])
                    y_train = train_df['totalEsg']
                    X_test  = test_df.drop(columns=['totalEsg'])
                    y_test  = test_df['totalEsg']
                    
                    # Train and evaluate models
                    for model_name, model in models.items():
                        # Prepare grid search
                        grid = GridSearchCV(
                            estimator=model,
                            param_grid=param_grids[model_name],
                            cv=3,
                            scoring='neg_root_mean_squared_error',
                            n_jobs=-1
                        )
                        # Tune on training data
                        grid.fit(X_train, y_train)
                        best_model = grid.best_estimator_

                        # train
                        # model.fit(X_train, y_train)


                        # predict
                        # y_pred = model.predict(X_test)
                        y_pred = best_model.predict(X_test)

                        # compute metrics
                        rmse, mae, r2 = compute_metrics(y_test, y_pred)
                        stability = compute_stability(best_model, X_test, y_test,
                                                  noise_level=0.01, n_trials=5)
                        
                        results.append({
                            'scenario_type': scenario_type,
                            'scenario': scenario_name,
                            'split': split_type,
                            'context': context_name,
                            'model': model_name,
                            'RMSE': rmse,
                            'MAE': mae,
                            'R2': r2,
                            'Stability': stability
                        })

results_df = pd.DataFrame(results)

Scenario: baselines, full, Split: within_sample
(291, 83) (73, 83)


l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.99506e-27): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\xgboost\training.py:183: UserWarning: [07:32:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Scenario: baselines, full, Split: region_holdout, Context: east_asia_n_pacific
(301, 82) (63, 82)


l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.96165e-27): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\xgboost\training.py:183: UserWarning: [07:32:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Scenario: baselines, full, Split: region_holdout, Context: europe_n_central_asia
(275, 82) (89, 82)


...

Scenario: diversified, median_balanced, Split: size_holdout, Context: Mid-Cap
(275, 83) (125, 83)


l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=2.0202e-27): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\xgboost\training.py:183: UserWarning: [07:36:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Scenario: diversified, median_balanced, Split: size_holdout, Context: Small-Cap
(275, 83) (125, 83)


l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=2.01205e-27): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\xgboost\training.py:183: UserWarning: [07:36:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


CPU times: total: 3min 51s
Wall time: 4min 14s


In [15]:
results_df = pd.DataFrame(results)

In [16]:
results_df

,scenario_type,scenario,split,context,model,RMSE,MAE,R2,Stability
0,baselines,full,within_sample,within_sample,Ridge,4.259572e+01,1.485193e+01,-2.906026e+01,0.001229
1,baselines,full,within_sample,within_sample,RandomForest,6.853874e+00,5.183024e+00,2.217250e-01,0.048605
2,baselines,full,within_sample,within_sample,XGBoost,6.757275e+00,4.804277e+00,2.435085e-01,0.114090
3,baselines,full,within_sample,within_sample,MLP,1.995883e+08,3.212502e+07,-6.599803e+14,0.000536
4,baselines,full,region_holdout,east_asia_n_pacific,Ridge,1.787572e+02,7.654021e+01,-5.621596e+02,-0.001267
...,...,...,...,...,...,...,...,...,...
135,diversified,median_balanced,size_holdout,Mid-Cap,MLP,1.481721e+09,2.293669e+08,-4.045075e+16,-0.000275
136,diversified,median_balanced,size_holdout,Small-Cap,Ridge,1.808906e+01,1.144357e+01,-3.088830e+00,0.000232
137,diversified,median_balanced,size_holdout,Small-Cap,RandomForest,8.895846e+00,6.472343e+00,1.112487e-02,-0.007204
138,diversified,median_balanced,size_holdout,Small-Cap,XGBoost,9.466702e+00,6.653255e+00,-1.198616e-01,-0.010946


In [17]:
results_df['R2'].max()

0.8708870271267248

In [ ]:
# 1. Extract within-sample RMSE per scenario/model
within = results_df[results_df['split'] == 'within_sample'][['scenario_type', 'scenario', 'model', 'RMSE']]
within = within.rename(columns={'RMSE': 'RMSE_within'})

# 2. Merge to get RMSE_within alongside all rows
merged = results_df.merge(within, on=['scenario_type', 'scenario', 'model'], how='left')

# 3. Compute Cross-Context Generalization Score:
#    Transfer Score = 1 - (RMSE_holdout / RMSE_within)
#    For within-sample rows, set NaN
merged['CrossContextScore'] = np.where(
    merged['split'] == 'within_sample',
    np.nan,
    1 - merged['RMSE'] / merged['RMSE_within']
)